## Titanic Survival Prediction Notebook
In this notebook, I use decision tree to predict whether a passenger is survived or not based on some of features:
* Embarked
* Cabin
* Fare
* Parch
* SibSp

In this excersise, I try to use the algorithm called **Iterative Dichotomiser 3** to build the decision tree myself. This algorithm uses _Entropy_ function and _Information gain_ as metrics.

In this algorithm, the best feature/attribute is chosen based on the information gained. The feature which has the highest information gain is used to split the dataset.

#### The formula to calculte the entropy:
![](https://cdn-images-1.medium.com/max/2000/1*EoWJ8bxc-iqBS-dF-XxsBA.jpeg)

#### The formula to calculate the information gain:
![](https://cdn-images-1.medium.com/max/2000/1*wQjVzx7zCVb87htqk46vUA.jpeg)

#### The steps to apply **Iterative Dichotomiser 3** to build the decision tree:
1. Compute the entropy for data-set
2. For every attribute/feature:
    1. Calculate entropy for all categorical values
    2. Take average information entropy for the current attribute
    3. Calculate gain for the current attribute
3. Pick the highest gain attribute.
4. Repeat until we get the tree we desired.


First of all, before building the decision tree, we have to analyze and clean the dataset before use it to train.

### Load libraries

In [1]:
import operator
import pandas as pd
import math as math

### Read the data

In [36]:
in_file = 'train.csv'
full_data = pd.read_csv(in_file)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### Analyze the data

As we can see, the column **PassengerId**, **Ticket** and **Name** should be removed because it is different for each passenger.
We also remove the column **Sex**, **Age**, **PClass** according the requirement of the instructor.

In [37]:
full_data = full_data.drop('PassengerId', axis=1)
full_data = full_data.drop('Ticket', axis=1)
full_data = full_data.drop('Pclass', axis=1)
full_data = full_data.drop('Age', axis=1)
full_data = full_data.drop('Sex', axis=1)
full_data = full_data.drop('Name', axis=1)
full_data.head()

,Survived,SibSp,Parch,Fare,Cabin,Embarked
0,0,1,0,7.2500,NaN,S
1,1,1,0,71.2833,C85,C
2,1,0,0,7.9250,NaN,S
3,1,1,0,53.1000,C123,S
4,0,0,0,8.0500,NaN,S


It is also clearly seen that some rows has the value _NaN_ at column **Cabin** so we will replace all the value _NaN_ with _U_ which means _Unknown_

In [38]:
full_data['Cabin'].fillna('U', inplace=True)
full_data.head()

,Survived,SibSp,Parch,Fare,Cabin,Embarked
0,0,1,0,7.2500,U,S
1,1,1,0,71.2833,C85,C
2,1,0,0,7.9250,U,S
3,1,1,0,53.1000,C123,S
4,0,0,0,8.0500,U,S


The data of column **Fare** is continuous variable so we have to convert data into discrete variable. 

In [39]:
print full_data['Fare'].max()
print full_data['Fare'].min()

512.3292
0.0


We find out that the max value of **Fare** is 512.3292 and the minimum value is 0. Therefore, I decide divide the range of the fare into 6 and each separates by 100 so that the **Fare** column is discrete and only contains the value from 1 to 6.

In [40]:
full_data.loc[operator.__and__(0.0 <= full_data.Fare, full_data.Fare <= 100.0), 'Fare'] = 1
full_data.loc[operator.__and__(101.0 <= full_data.Fare, full_data.Fare <= 200.0), 'Fare'] = 2
full_data.loc[operator.__and__(201.0 <= full_data.Fare, full_data.Fare <= 300.0), 'Fare'] = 3
full_data.loc[operator.__and__(301.0 <= full_data.Fare, full_data.Fare <= 400.0), 'Fare'] = 4
full_data.loc[operator.__and__(401.0<= full_data.Fare, full_data.Fare <= 500.0), 'Fare'] = 5
full_data.loc[501.0 <= full_data.Fare, 'Fare'] = 6
full_data.head()

,Survived,SibSp,Parch,Fare,Cabin,Embarked
0,0,1,0,1.0,U,S
1,1,1,0,1.0,C85,C
2,1,0,0,1.0,U,S
3,1,1,0,1.0,C123,S
4,0,0,0,1.0,U,S


With the **Cabin** column, we should convert it into categorical variable

In [41]:
full_data['Cabin'] = full_data['Cabin'].apply(lambda cabin : cabin[:1])
full_data.head()

,Survived,SibSp,Parch,Fare,Cabin,Embarked
0,0,1,0,1.0,U,S
1,1,1,0,1.0,C,C
2,1,0,0,1.0,U,S
3,1,1,0,1.0,C,S
4,0,0,0,1.0,U,S


After cleaning up the data, I divide into two set, training dataset and testing data set with the ratio of 7:3

In [42]:
original_data = full_data.sample(frac=0.7)
test_data = full_data.drop(original_data.index)

I store my whole decision tree in a variable call _tree_.
To automate performing validation on my testing data set, I create a class called **Node** which is used while classify a test-instance using the tree which was built earlier.

In [43]:
tree = {}

In [44]:
class Node():
    value = ""
    children = []

    def __init__(self, val, dictionary):
        self.value = val
        if (isinstance(dictionary, dict)):
            self.children = dictionary.keys()

### Building Tree

The function _find_tree_ is used to build the tree based on the **ID3** algorithm based on the steps described previous. The function _calculate_entropy_ is used to calculate the entropy of each set.

In [48]:
def find_tree(original_data):
    count_survived = original_data[original_data['Survived'] == 1].shape[0]
    count_not_survived = original_data[original_data['Survived'] == 0].shape[0]
    if count_survived == 0 and count_not_survived != 0:
        return 0
    if count_not_survived == 0 and count_survived != 0:
        return 1
    list_features = list(original_data.columns.values)
    list_features.remove('Survived')
    max_feature = None
    max_gain = -1
    for feature in list_features:
        list_values = original_data[feature].unique()
        entropy = 0
        for value in list_values:
            entropy += calculate_entropy(original_data, feature, value)
        gain = calculate_entropy(original_data, feature=None, value=None) - entropy
        if gain > max_gain:
            max_feature = feature
            max_gain = gain
    tree = {max_feature: {}}
    for value in original_data[max_feature].unique():
        new_data = original_data.copy()
        new_data = new_data[new_data[max_feature] == value]
        new_data = new_data.drop(max_feature, axis=1)
        features = list(new_data.columns.values)
        if len(features) == 1 and features.__contains__('Survived'):
            if count_not_survived > count_survived:
                tree[max_feature][value] = 0
            else:
                tree[max_feature][value] = 1
            continue
        subtree = find_tree(new_data)
        tree[max_feature][value] = subtree
    return tree

def calculate_entropy(original_data, feature, value):
    data = original_data.copy()
    if feature != None and value != None:
        data = data[data[feature] == value]
    # count row
    # data.shape[1] count columns
    total = (float)(data.shape[0])
    num_survived = (float)(data[data['Survived'] == 1].shape[0])
    num_not_survived = (float)(total - num_survived)
    if num_survived == 0.0 or num_not_survived == 0.0:
        return 0
    pyes = -(num_survived / total) * math.log(num_survived / total, 2)
    pno = -(num_not_survived / total) * math.log(num_not_survived / total, 2)
    pt = total / original_data.shape[0]
    return (pyes + pno) * pt

In [49]:
tree = find_tree(original_data)

The function _accuracy_score_ is used to calculate the accuracy of predicted value compared with the test data.

In [50]:
def accuracy_score(truth, pred, name):
    """ Returns accuracy score for input truth and predictions. """

    if len(truth) == len(pred):
        # Calculate and return the accuracy as a percent
        # {:2f}.format((truth == pred).mean()*100)
        # ":" represents format specification
        # "2f" represents 2 decimal places
        return "{} Predictions have an accuracy of {:.2f}.".format(name, (truth.values == pred.values).mean() * 100)
    else:
        return "Number of predictions does not match number of outcomes!"

In [51]:
outcomes = test_data['Survived']

The code below is used to use the tree to predict whether a passenger is survived or not and calculate the accuracy of the tree

In [52]:
results = []
for entry in test_data.iterrows():
    tempDict = tree.copy()
    result = ""
    while (isinstance(tempDict, dict)):
        root = Node(tempDict.keys()[0], tempDict[tempDict.keys()[0]])
        tempDict = tempDict[tempDict.keys()[0]]
        value = entry[1][root.value]
        if (value in tempDict.keys()):
            child = Node(value, tempDict[value])
            result = tempDict[value]
            tempDict = tempDict[value]
        else:
            result = "Null"
            break
    if result != "Null":
        results.append(result)
    else:
        results.append(0)
results = pd.Series(results)
outcomes = test_data['Survived']
results.reset_index(drop=True)
outcomes.reset_index(drop=True)
print(accuracy_score(outcomes, results, "Accuracy"))

Accuracy Predictions have an accuracy of 66.29.


The accuracy of 66% is quite low so I extract titles from passenger names. In this case, I do not drop the **Name** column but I convert it into the categorical variable. The map to map the title of each passenger is described below.

In [53]:
Title_Dictionary = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"

                    }

After converting the **Name** column, I run my code to build the tree again and I achieve the accuracy of **79%**. 

You can get the whole code [here](https://github.com/vtnlinh95/Titanic_passenger_surrvival_prediction/blob/master/TitanicPrediction.py)